#DCC 2023 MISSION 3

In [ ]:
# 필요한 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np


# GPU를 사용 가능한 경우 GPU를, 그렇지 않은 경우 CPU를 사용합니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Local에 있는 넘파이배열 형태의 테이터를 Torch tensor형태로 변환합니다.

In [ ]:
# npy 파일에서 데이터 로드
x_data = np.load('x_data.npy')
y_data = np.load('y_data.npy')
xTest = np.load('xTest.npy')
yTest = np.load('yTest.npy')

# 데이터를 훈련 세트와 검증 세트로 나누기
def train_val_split(x_data, y_data, split_ratio=0.8, random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)
    
    num_samples = len(x_data)
    num_train_samples = int(num_samples * split_ratio)
    
    # 데이터를 섞은 후 나누기
    indices = np.random.permutation(num_samples)
    train_indices = indices[:num_train_samples]
    val_indices = indices[num_train_samples:]
    
    xTrain = x_data[train_indices]
    yTrain = y_data[train_indices]
    xValidation = x_data[val_indices]
    yValidation = y_data[val_indices]
    
    return xTrain, yTrain, xValidation, yValidation

xTrain, yTrain, xValidation, yValidation = train_val_split(x_data, y_data, split_ratio=0.8, random_seed=42)

# Triain, Validation, test 데이터를 PyTorch 텐서로 변환하고 전처리합니다.
xTrain = torch.from_numpy(xTrain)
yTrain = torch.from_numpy((yTrain == 0).astype(np.int))

xValidation = torch.from_numpy(xValidation)
yValidation = torch.from_numpy((yValidation == 0).astype(np.int))

xTest = torch.from_numpy(xTest)
yTest = torch.from_numpy((yTest == 0).astype(np.int))

# 데이터로더 생성
batch_size = 128  # 원하는 배치 크기로 설정
train_dataset = CustomDataset(xTrain, yTrain, transform=transform)
validation_dataset = CustomDataset(xValidation, yValidation, transform=transform)
test_dataset = CustomDataset(xTest, yTest, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## 모델 클래스 정의

In [ ]:
class BinaryModel(nn.Module):
    def __init__(self):
        super(BinaryModel, self).__init__()

        # 합성곱 레이어: 이미지의 특징 추출
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3)
        
        # 최대 풀링 레이어: 공간 해상도 감소
        self.pool = nn.MaxPool2d(2)
        
        # 완전 연결 레이어 1: 특징을 추출한 후 중간 표현 생성
        self.fc1 = nn.Linear(128, 128)
        
        # 완전 연결 레이어 2: 중간 표현을 더 복잡한 형태로 변환
        self.fc2 = nn.Linear(128, 256)
        
        # 완전 연결 레이어 3: 이진 분류를 위한 출력 생성
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x):
        # 합성곱 레이어와 활성화 함수 적용
        x = self.pool(torch.relu(self.conv1(x)))
        
        # 출력을 1차원 벡터로 평평하게 만듦
        x = x.view(x.size(0), -1)
        
        # 완전 연결 레이어와 ReLU 활성화 함수 적용
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        
        # 이진 분류를 위한 출력 레이어
        x = self.fc3(x)
        
        return x

In [ ]:
# 모델 인스턴스 생성 및 GPU 설정
model = BinaryModel()
model.to(device)

In [ ]:
# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

## Train

In [ ]:
num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()
        optimizer.zero_grad()
        outputs = model(inputs).view(-1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"에폭 {epoch + 1}/{num_epochs}, 손실: {running_loss / len(train_loader)}")

## validation

In [ ]:
model.eval()
validation_losses = []  # 검증 손실을 저장할 리스트

with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()
        outputs = model(inputs).view(-1)
        loss = criterion(outputs, labels)
        validation_losses.append(loss.item())

In [ ]:
# 훈련된 모델을 저장합니다.
torch.save(model.state_dict(), "binary_model.pt")

## Test

In [ ]:
model.eval()

# 예측을 저장할 빈 리스트를 생성
predictions = []

with torch.no_grad():
    for i in range(0, len(xTest), batch_size):
        batch_x = xTest[i:i + batch_size]
        batch_y = yTest[i:i + batch_size]
        batch_x = batch_x.to(device)
        outputs = model(batch_x).view(-1)
        predicted = (outputs > 0.5).float()
        predictions.extend(predicted.cpu().numpy())

In [ ]:
accuracy = accuracy_score(yTest, predictions)
precision = precision_score(yTest, predictions)
recall = recall_score(yTest, predictions)
f1 = f1_score(yTest, predictions)
conf_matrix = confusion_matrix(yTest, predictions)
fpr, tpr, thresholds = roc_curve(yTest, predictions)
roc_auc = auc(fpr, tpr)

## Graph

In [ ]:
# 각 평가 항목을 그래프로 시각화
metrics = ["Accuracy", "Precision", "Recall", "F1 Score"]
values = [accuracy, precision, recall, f1]

plt.figure(figsize=(10, 6))
plt.bar(metrics, values, color='royalblue')
for i in range(len(metrics)):
    plt.text(i, values[i], f"{values[i]:.2f}", ha='center', va='bottom')
plt.title("평가 항목 그래프")
plt.xlabel("평가 항목")
plt.ylabel("값")
plt.show()